In [1]:
import torch
from pathlib import Path
import argparse

In [23]:
import pandas as pd
df = pd.DataFrame({'num_legs': [2, 4, 8, 0],
                'num_wings': [2, 0, 0, 0],
                'num_specimen_seen': [10, 2, 1, 8]},
                index=['falcon', 'dog', 'spider', 'fish'])

In [2]:
import numpy as np

In [8]:
set(torch.from_numpy(np.array([1,0,1,0,1,0,0,1])).cuda().long().cpu().numpy())

{0, 1}

In [29]:

df.sample(n=min(2, len(df)), random_state=1)

,num_legs,num_wings,num_specimen_seen
fish,0,0,8
spider,8,0,1


In [43]:
class Model(torch.nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = torch.nn.Linear(input_dim, 2)
        self.layer2 = torch.nn.Linear(2, 2)
        self.layer3 = torch.nn.Linear(2, 2)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x
    
    def hidden(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return x

In [70]:
_model = Model(2)

In [45]:
_model

Model(
  (layer1): Linear(in_features=2, out_features=2, bias=True)
  (layer2): Linear(in_features=2, out_features=2, bias=True)
  (layer3): Linear(in_features=2, out_features=2, bias=True)
)

In [46]:
from torch import optim

In [58]:
parameters = [
    {"params":_model.layer1.parameters()},
    {"params":_model.layer2.parameters(),}
]

In [71]:
optimizer = optim.Adam(_model.parameters())

In [72]:
# loss = torch.mean(_model(torch.randn(10, 2)))

_hidden = _model.hidden(torch.randn(10, 2)).detach()


loss = torch.mean(_model.layer3(_hidden))
loss.backward()

for name, param in _model.named_parameters():
    print(name, param.grad)

layer1.weight None
layer1.bias None
layer2.weight None
layer2.bias None
layer3.weight tensor([[-0.0660, -0.0946],
        [-0.0660, -0.0946]])
layer3.bias tensor([0.5000, 0.5000])


In [74]:
optimizer.step()
optimizer.zero_grad()

In [75]:
for name, param in _model.named_parameters():
    print(name, param.grad)

layer1.weight None
layer1.bias None
layer2.weight None
layer2.bias None
layer3.weight tensor([[0., 0.],
        [0., 0.]])
layer3.bias tensor([0., 0.])


In [63]:
zero_grad_parameters = [
    {"params":_model.layer3.parameters(),}
]

In [64]:
def zero_grad(param_groups):
    r"""Clears the gradients of all optimized :class:`torch.Tensor` s."""
    for group in param_groups:
        for p in group['params']:
            if p.grad is not None:
                p.grad.detach_()
                p.grad.zero_()

In [65]:
zero_grad(zero_grad_parameters)

In [66]:
for name, param in _model.named_parameters():
    print(name, param.grad)

layer1.weight tensor([[-0.0037,  0.0078],
        [ 0.0003, -0.0007]])
layer1.bias tensor([ 0.0443, -0.0038])
layer2.weight tensor([[-0.0349,  0.0052],
        [ 0.1550, -0.0230]])
layer2.bias tensor([-0.1076,  0.4787])
layer3.weight tensor([[0., 0.],
        [0., 0.]])
layer3.bias tensor([0., 0.])


In [78]:
11 % 3

2

In [22]:
import shutil

In [5]:
vars = {
    "asd":1
}

In [8]:
import yaml

In [11]:
with open(r"D:\Project\Fair_NLP_Classification\dev\results\dev\Moji\dev_fcl\opt.yaml", 'r') as f:
    f = yaml.full_load(f)

In [18]:
argparse.Namespace(**f)

Namespace(BT=None, BTObj=None, DyBT=None, DyBTObj=None, DyBTalpha=0.1, DyBTinit='original', FCL=True, INLP=False, INLP_by_class=False, INLP_discriminator_reweighting=None, INLP_min_acc=0.0, INLP_n=300, activation_function='Tanh', adv_BT=None, adv_BTObj=None, adv_activation_function='ReLu', adv_batch_norm=False, adv_batch_size=1024, adv_checkpoint_interval=1, adv_debiasing=False, adv_decoupling=False, adv_diverse_lambda=0.0, adv_dropout=0, adv_epochs=100, adv_epochs_since_improvement=5, adv_gated=False, adv_gated_mapping=None, adv_hidden_size=300, adv_lambda=1, adv_level='last_hidden', adv_lr=0.001, adv_n_hidden=2, adv_num_classes=2, adv_num_subDiscriminator=1, adv_test_batch_size=1024, adv_uniform_label=False, adv_update_frequency='Batch', base_seed=1, batch_norm=False, batch_size=1024, checkpoint_interval=1, data_dir=None, dataset='Moji', device_id=0, dropout=0, emb_size=2304, encoder_architecture='Fixed', epochs=100, epochs_since_improvement=5, exp_id='dev_fcl', fcl_base_temperature_

In [15]:
f.keys()

dict_keys(['BT', 'BTObj', 'DyBT', 'DyBTObj', 'DyBTalpha', 'DyBTinit', 'FCL', 'INLP', 'INLP_by_class', 'INLP_discriminator_reweighting', 'INLP_min_acc', 'INLP_n', 'activation_function', 'adv_BT', 'adv_BTObj', 'adv_activation_function', 'adv_batch_norm', 'adv_batch_size', 'adv_checkpoint_interval', 'adv_debiasing', 'adv_decoupling', 'adv_diverse_lambda', 'adv_dropout', 'adv_epochs', 'adv_epochs_since_improvement', 'adv_gated', 'adv_gated_mapping', 'adv_hidden_size', 'adv_lambda', 'adv_level', 'adv_lr', 'adv_n_hidden', 'adv_num_classes', 'adv_num_subDiscriminator', 'adv_test_batch_size', 'adv_uniform_label', 'adv_update_frequency', 'base_seed', 'batch_norm', 'batch_size', 'checkpoint_interval', 'data_dir', 'dataset', 'device_id', 'dropout', 'emb_size', 'encoder_architecture', 'epochs', 'epochs_since_improvement', 'exp_id', 'fcl_base_temperature_g', 'fcl_base_temperature_y', 'fcl_lambda_g', 'fcl_lambda_y', 'fcl_temperature_g', 'fcl_temperature_y', 'full_label', 'gated', 'gated_mapping', 'h

In [17]:
type(f["adv_debiasing"])

bool

In [21]:
_tmp_dict = {"1":1}
_tmp_dict.update({"2":2})
print(_tmp_dict)

{'1': 1, '2': 2}


In [6]:
vars

{'asd': 1}

In [7]:
argparse.Namespace(**vars)

Namespace(asd=1)

In [3]:
argparse.Namespace()

TypeError: vars() argument must have __dict__ attribute

In [4]:
vars({"K":1})

TypeError: vars() argument must have __dict__ attribute

In [2]:
checkpoint_dir = Path(r"logs\Bios_both\both_adv\models")

In [3]:
_epoch_results = torch.load(checkpoint_dir / "checkpoint_epoch18.pth.tar")

In [4]:
_epoch_results.keys()

dict_keys(['epoch', 'epochs_since_improvement', 'loss', 'dev_predictions', 'test_predictions', 'dev_evaluations', 'test_evaluations'])

In [5]:
_epoch_results["dev_evaluations"]

{'rms_TPR': 0.43560272485196483,
 'accuracy': 0.8031215772179627,
 'macro_fscore': 0.7283969094156552,
 'micro_fscore': 0.8031215772179627}

In [6]:
# Load Bios dataset
import pandas as pd

data_dir = Path(r"D:\Project\Minding_Imbalance_in_Discriminator_Training\data\bios")

bios_dev_df = pd.read_pickle(data_dir / "bios_dev_df.pkl")
bios_test_df = pd.read_pickle(data_dir / "bios_test_df.pkl")

In [7]:
def return_y_g(my_df):
    my_df = my_df[my_df["economy_label"]!="Unknown"]
    return my_df["profession_class"].to_numpy(), my_df["intersection_class"].to_numpy()

In [8]:
dev_y, dev_g = return_y_g(bios_dev_df)
test_y, test_g = return_y_g(bios_test_df)

In [11]:
from src.evaluator import gap_eval_scores

In [12]:
gap_eval_scores(
    y_pred = _epoch_results["dev_predictions"], 
    y_true = dev_y, 
    protected_attribute = dev_g)

{'accuracy': 0.8031215772179627,
 'macro_fscore': 0.7283969094156552,
 'micro_fscore': 0.8031215772179627,
 'TPR_GAP': 0.43560272485196483,
 'FPR_GAP': 0.005944258843174327,
 'PPR_GAP': 0.039407736760176205}

In [13]:
gap_eval_scores(
    y_pred = _epoch_results["test_predictions"], 
    y_true = test_y, 
    protected_attribute = test_g)

{'accuracy': 0.8068291982334566,
 'macro_fscore': 0.7401184913339307,
 'micro_fscore': 0.8068291982334566,
 'TPR_GAP': 0.4340048368332264,
 'FPR_GAP': 0.006683039984137971,
 'PPR_GAP': 0.04327407716322931}

In [1]:
import torch

In [6]:
torch.cat([torch.rand((3,2)), torch.rand((4,2))], dim=0)

tensor([[0.3813, 0.4044],
        [0.5354, 0.5314],
        [0.2815, 0.2595],
        [0.1855, 0.4117],
        [0.6113, 0.1589],
        [0.0065, 0.0567],
        [0.2697, 0.1700]])

In [7]:
items = [0,1,2,3,4,5,6,7,8]

lb_dict = {
    (1,item):i for i, item in enumerate(items)
}

In [8]:
lb_dict

{(1, 0): 0,
 (1, 1): 1,
 (1, 2): 2,
 (1, 3): 3,
 (1, 4): 4,
 (1, 5): 5,
 (1, 6): 6,
 (1, 7): 7,
 (1, 8): 8}

In [10]:
[lb_dict[(1, item)]+=1 for item in items]

lb_dict

SyntaxError: invalid syntax (Temp/ipykernel_27468/1820016294.py, line 1)

In [13]:
import numpy as np
np.array([1,2,3]).shape

(3,)

In [17]:
tt = torch.tensor(
    [
        [
        [1,2,3],
        [4,5,6]
    ]]
)

In [18]:
tt

tensor([[[1, 2, 3],
         [4, 5, 6]]])

In [19]:
tt.squeeze()

tensor([[1, 2, 3],
        [4, 5, 6]])